<a href="https://colab.research.google.com/github/liudiepie/Introduction_to_Machine_Learning/blob/main/Assignment_Multi_Layer_Perceptrons_(MLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Layer Perceptrons (MLP)

In [121]:
import torch

x = torch.rand(5, 3)*2 - 1
x_relu_max = torch.max(torch.zeros_like(x),x)

print("x: {}".format(x))
print("x after ReLU with max: {}".format(x_relu_max))

x: tensor([[ 0.1779, -0.5660, -0.7355],
        [-0.2317, -0.7305,  0.1393],
        [-0.2435,  0.6126,  0.8400],
        [-0.6209, -0.7020,  0.6116],
        [ 0.6465,  0.1652, -0.5081]])
x after ReLU with max: tensor([[0.1779, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1393],
        [0.0000, 0.6126, 0.8400],
        [0.0000, 0.0000, 0.6116],
        [0.6465, 0.1652, 0.0000]])


In [122]:
import torch.nn.functional as F

x_relu_F = F.relu(x)

print("x after ReLU with nn.funtional: {}".format(x_relu_F))

x after ReLU with nn.funtional: tensor([[0.1779, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1393],
        [0.0000, 0.6126, 0.8400],
        [0.0000, 0.0000, 0.6116],
        [0.6465, 0.1652, 0.0000]])


In [123]:
import numpy as np
from torch import nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm import tqdm
from tqdm import tqdm_notebook


In [124]:
# Load the data
mnist_train = datasets.MNIST(root="./datasets",train = True, transform = transforms.ToTensor(), download = True)
mnist_test = datasets.MNIST(root="./datasets",train = False, transform = transforms.ToTensor(), download = True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle = True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle = False)

### Training

In [125]:
# Define the MLP function
class MLP(nn.Module):
     def __init__(self):
         super(MLP, self).__init__()
         self.fc1 = nn.Linear(28*28, 500)
         self.fc2 = nn.Linear(500, 10)
         self.layers = nn.Sequential(nn.Linear(28*28, 500), nn.ReLU(), nn.Linear(500, 10))

         
     def forward(self, x):
         x = F.relu(self.fc1(x))
         x = self.fc2(x)
         return x
 
mlp = MLP()

In [135]:
# Optimizer
optimizer = torch.optim.SGD(mlp.parameters(), lr=0.1)

In [136]:
# Iterate through train set minibatchs
for images, labels in tqdm(train_loader):
  # Zero out the gradients
  optimizer.zero_grad()
  mlp.train()
  images = Variable(images)
  labels = Variable(labels)
  # Forward pass
  x = images.view(-1, 28*28)
  out = mlp(x)
  cross_entropy = F.cross_entropy(out, labels)
  # Backward pass
  cross_entropy.backward()
  optimizer.step()

100%|██████████| 600/600 [00:11<00:00, 52.07it/s]


### Testing

In [137]:
correct = 0
total = len(mnist_test)

# Iterate through test set minibatchs
with torch.no_grad():
  for images, labels in tqdm(test_loader):
    # Zero out the gradients
    optimizer.zero_grad()
    mlp.train()
    images = Variable(images)
    labels = Variable(labels)
    # Forward pass
    x = images.view(-1, 28*28)
    out = mlp(x)

    predictions = torch.argmax(out, dim = 1)
    correct += torch.sum((predictions == labels).float())

print('Test accuracy: {}'.format(correct/total))

100%|██████████| 100/100 [00:01<00:00, 62.20it/s]

Test accuracy: 0.9236999750137329
